<font size="5">Performance Measurement UltraTrail with different Clock-Mode Settings</font>

In this notebook the different clock mode settings of the Akida neural processor from BrainChip is investigated.

In [1]:
import os
from os import listdir
from os.path import isdir, join
import pathlib

import akida
from akida import FullyConnected
from akida import evaluate_sparsity
import cnn2snn
from cnn2snn import check_model_compatibility
from cnn2snn import quantize
from cnn2snn import quantize_layer
from cnn2snn import convert

from math import ceil

from timeit import default_timer as timer
import numpy as np

In [2]:
# Load the quantized CNN

quantized_model = cnn2snn.load_quantized_model('final_quantized_ultratrail_model.h5')

In [3]:
#  Convert to an Akida model

akida_model = convert(quantized_model)
akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 12]    1          10    
_______________________________________________

                SW/conv_0-dense (Software)                 
___________________________________________________________
Layer (type)             Output shape   Kernel shape     
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)    
___________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)    
___________________________________________________________
                                        (1, 1, 32, 32)   
___________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)    
___________________________________________________________
                                        (1, 1, 32, 64)   
_______________________________________________________

In [4]:
# Load the test set

feature_sets_filename = 'final1_stored_files_targets_int_normalized.npz'

feature_sets = np.load(feature_sets_filename)

print('Feature Sets: ', feature_sets.files)

Feature Sets:  ['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test']


In [5]:
# Assign test set

x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

In [6]:
# Add dimension

x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)

print(x_test.shape)

(4371, 40, 101, 1)


In [7]:
# Define device 

devices = akida.devices()
print('Akida NP available :', devices[0])

Akida NP available : <akida.core.HardwareDevice object at 0xffff457229b0>


In [8]:
# Print device version

print(devices[0].version)

BC.00.000.002


In [9]:
# Map Akida model to device

device = devices[0]

akida_model.map(device)

akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 12]    3          10    
_______________________________________________

     HW/conv_0-separable_7 (Hardware) - size: 204356 bytes     
_______________________________________________________________
Layer (type)             Output shape   Kernel shape      NPs
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)     N/A
_______________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 32)       
_______________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 64)       
_______

<font size="5">Clock Mode: Performance</font>

In the following are the power measurements with the clock mode "Performance" enabled. 

In [10]:
device.soc.power_measurement_enabled = True
device.soc.clock_mode = akida.soc.ClockMode.Performance

In [13]:
start = timer()
akida_model.forward(x_test)
end = timer()
# Display floor current
floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 900.01 mW

Sequence HW/conv_0-separable_7
Average framerate = 383.42 fps
Last inference power range (mW):  Avg 1032.91 / Min 1032.00 / Max 1034.00 / Std 1.02 
Last inference energy consumed (mJ/frame): 2.69
Sequence SW/separable_8
Average framerate = 1096.59 fps
Last inference power range (mW):  Avg 905.21 / Min 902.00 / Max 974.00 / Std 13.37 
Last inference energy consumed (mJ/frame): 0.83
Sequence HW/dense
Average framerate = 3216.34 fps
Last inference power range (mW):  Avg 904.28 / Min 902.00 / Max 917.00 / Std 4.65 
Last inference energy consumed (mJ/frame): 0.28
Akida inference 4371 MFCC took 16.81 s.



In [16]:
print(x_test[0].shape)

(40, 101, 1)


In [19]:
inf_test = inf_test.reshape(1, 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)

print(inf_test.shape)

(1, 40, 101, 1)


In [21]:
start = timer()
akida_model.forward(inf_test)
end = timer()
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(inf_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 900.01 mW

Sequence HW/conv_0-separable_7
Average framerate = 76.92 fps
Sequence SW/separable_8
Average framerate = 142.86 fps
Sequence HW/dense
Average framerate = 1000.00 fps
Akida inference 1 MFCC took 0.05 s.



<font size="5">Clock Mode: Economy</font>

In the following are the power measurements with the clock mode "Economy" enabled. 

In [22]:
device.soc.clock_mode = akida.soc.ClockMode.Economy
print(device.soc.clock_mode)

ClockMode.Economy


In [23]:
start = timer()
akida_model.forward(x_test)
end = timer()
# Display floor current
floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 735.69 mW

Sequence HW/conv_0-separable_7
Average framerate = 127.84 fps
Last inference power range (mW):  Avg 780.00 / Min 780.00 / Max 780.00 / Std 0.00 
Last inference energy consumed (mJ/frame): 6.10
Sequence SW/separable_8
Average framerate = 1086.50 fps
Last inference power range (mW):  Avg 736.23 / Min 735.00 / Max 780.00 / Std 6.36 
Last inference energy consumed (mJ/frame): 0.68
Sequence HW/dense
Average framerate = 3142.34 fps
Last inference power range (mW):  Avg 735.33 / Min 735.00 / Max 738.00 / Std 0.97 
Last inference energy consumed (mJ/frame): 0.23
Akida inference 4371 MFCC took 39.68 s.



In [24]:
start = timer()
akida_model.forward(inf_test)
end = timer()
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(inf_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 735.69 mW

Sequence HW/conv_0-separable_7
Average framerate = 27.78 fps
Sequence SW/separable_8
Average framerate = 142.86 fps
Sequence HW/dense
Average framerate = 500.00 fps
Akida inference 1 MFCC took 0.08 s.



<font size="5">Clock Mode: LowPower</font>

In the following are the power measurements with the clock mode "LowPower" enabled. 

In [25]:
device.soc.clock_mode = akida.soc.ClockMode.LowPower
print(device.soc.clock_mode)

ClockMode.LowPower


In [26]:
start = timer()
akida_model.forward(x_test)
end = timer()
# Display floor current
floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 653.35 mW

Sequence HW/conv_0-separable_7
Average framerate = 6.39 fps
Sequence SW/separable_8
Average framerate = 944.67 fps
Last inference power range (mW):  Avg 653.00 / Min 653.00 / Max 653.00 / Std 0.00 
Last inference energy consumed (mJ/frame): 0.69
Sequence HW/dense
Average framerate = 773.90 fps
Last inference power range (mW):  Avg 653.51 / Min 653.00 / Max 655.00 / Std 0.88 
Last inference energy consumed (mJ/frame): 0.84
Akida inference 4371 MFCC took 694.12 s.



In [27]:
start = timer()
akida_model.forward(inf_test)
end = timer()
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(inf_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 653.35 mW

Sequence HW/conv_0-separable_7
Average framerate = 1.43 fps
Last inference power range (mW):  Avg 655.00 / Min 655.00 / Max 655.00 / Std 0.00 
Last inference energy consumed (mJ/frame): 458.50
Sequence SW/separable_8
Average framerate = 250.00 fps
Sequence HW/dense
Average framerate = 500.00 fps
Akida inference 1 MFCC took 0.77 s.

